# <span style="color: blue;">Intro</span>

The first part of the analysis is to see what happens when ABRicate is run with both the Resfinder and CARD database to identify the extent of difference in how each method identifies "TRANSMISSIBLE RESISTANCE GENES"
Note by the fact we are only looking at "TRANSMISSIBLE RESISTNACE GENES" CARD and Resfinder dbs should have all necessary genes in them

**For two programs to identify the same thing they have to identify everything identically at the protein level**

### Setup 

**Dependencies**
* Python 3
* Biopython

**Inputs - Resistance Databases**
* resfinder_20191001 (Primary)
* card_20191023
* resfinder_20180122
* resfinder_20170126




In [20]:
# Importing modules and looking at local files

# Python inbuilt
import json
import csv
import subprocess
import os

# Extra downloaded
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Data handling
import numpy as np
import pandas as pd

# File Structure
%ls

 Volume in drive E is Data
 Volume Serial Number is 9A84-AE69

 Directory of E:\users\timdavies\git_projects\bioinformatic_fake_news\db_analysis

16/04/2020  14:42    <DIR>          .
16/04/2020  14:42    <DIR>          ..
02/04/2020  19:12    <DIR>          .ipynb_checkpoints
02/04/2020  19:12    <DIR>          card_20191023
02/04/2020  19:12         2,607,393 card_20191023_formatted.fasta
02/04/2020  19:12         2,566,359 card_20191023_link.csv
02/04/2020  19:12           264,362 db_comparison.csv
02/04/2020  19:12         8,010,987 db_comparison_full.json
14/04/2020  20:59    <DIR>          db_comparison_result_tarballs
16/04/2020  14:42            26,930 db_related.ipynb
02/04/2020  19:12         3,012,602 resfinder.fasta
02/04/2020  19:12    <DIR>          resfinder_20170126
02/04/2020  19:12    <DIR>          resfinder_20180122
02/04/2020  19:12    <DIR>          resfinder_20190122
02/04/2020  19:12    <DIR>          resfinder_20191001
02/04/2020  19:12    <DIR>          resfin

In [2]:
# First we PARSE the databases 
# CARD
card_db = "card_20191023/card-data/card.json"
with open(card_db) as f:
    card_db = json.load(f)

    

## <span style="color: blue;">Formatting and comparing the two databases</span>


So for comparing CARD with Resfinder results, were aiming to identify transmissible proteins from each
In card this approximately equates to the **protein homolog model**.

This process has several steps
1. For each database, create a csv which contains an easily accesible set of information for comparison
- Note for working out whether there is a translation link, we translate with translation table 11, then disregard the first element and remove any trailing stop codon sequence.
- Looking at the remaining sequence, if they match then there is a link, if not no link
- If no link leave blank, and each DNA with no link will be put next to its directly translated protein
2. As part of making a database each element will get a new identifier which will be used to make sure all programs can process them the same way
- e.g. cardnewid_x 
2. Match these CSVs according to protein sequence




#### The card CSV will have the following headers

1. prot_seq
2. dna_seq
3. card_name
4. card_newname
5. aro_id
6. translation_link

#### The resfinder CSV will have the following headers

1. prot_seq
2. dna_seq
3. resfinder_name
4. resfinder_newname


#### DATABASE QUALITY CONTROL

For each database, we need to create a unique identifier for elements
This based on its DNA sequence only
This in effect will remove a sequence which has the same DNA sequence but different name in any given database

e.g. supposed the following sequence AACTTGCTA was called both gene1 and gene2 in the formated databases it will only be called newgene1

Duplicates names for different sequences will be asigned new names. The aim of this approach is to retain the databases in as close to original format as possible , while making up to date databases readable for each of the 4 programs

##### Resfinder 1st October 2019 release

**Note the resfinder database is produced by concatinating each of the specific antibiotic databases, this results in some duplicates which we remove, (i.e. the genes which affect more than one type of antibiotic, (for example quinolone resistance causing aac variant)**

###### Removed variants below

So the duplicate names we will have censored using this method
1. "blaOXA-347_1_JN086160" , same sequence as "blaOXA-347_1_ACWG01000053" (2 refs)
2. "blaZ_129_CP003194", same sequence as "blaZ_125_CP003194" (curation)
3. "blaIMP-58_1_KU647281",  same sequence as "blaIMP-58_1_KU647281" (duplciate)
4. "blaCTX-M-63_1_AB205197", same sequence as "blaCTX-M-63_1_EU660216" (2 refs)
5. "blaCMY-110_1_AB872957", same sequence as "blaCMY-110_1_AB872957" (duplicate)
6. "blaCMY-104_1_KF150216", same sequence as "blaCMY-104_1_KF150216" (duplicate)
7. "blaACC-4_2_EF504260", same sequence as "blaACC-4_1_GU256641" (2 refs)
8. "blaSHV-36_1_AF467947", same sequence as "blaSHV-36_1_AF467947" (duplicate)
9. "blaOXA-60_1_AF525303", same sequence as "blaOXA-60_1_AF525303" (duplicate)
10. "blaFRI-1_1_KT192551", same sequence as "blaFRI-1_1_KT192551" (duplicate)
11. "cfr_1_AM408573", same sequence as "cfr_1_AM408573" (duplicate)
12. "cfr_2_AJ879565", same sequence as "cfr_2_AJ879565" (duplicate)
13. "cfr(B)_3_KR610408", same sequence as "cfr(B)_3_KR610408" (duplicate)
14. aac(6')-Ib-cr_1_DQ303918, same sequence as "aac(6')-Ib-cr_1_DQ303918" (duplicate)
15. aac(6')-Ib-cr_2_EF636461 , same sequence as "aac(6')-Ib-cr_2_EF636461" (duplicate)
16. dfrA22_3_FM957884, same sequence as dfrA33_1_FM957884 (curation)

##### CARD 23rd October 2019 release

This database is more complex in structure, but to attempt to produce a similar set of genes (i.e. just the transmissible genes) we only look at the genes which apply to the "protein homolog model"

In this database, there are several things to note which will be important as we compare the database:
* Each element has at least one DNA and protein sequence
* all of the DNA elements do translate to produce the related protein , but there is an issue with wether or not sequences include a stop codon and how the start codon is translated. Also some are not in the correct frame
* Also 1 sequence (Erm(44)v) has 2 sequences. This is not relavent to this study as it focusses on E. coli whereas this gene is only seen in S. saprophyticus and encodes macrolide resistance (to which E. coli is innately resistant)
* For simplicity here, we will just take the DNA sequences from the protein homolog CARD.
* Otherwise sequences in this database are all unique (see Resfinder Issues above)


#### DATABASE PREPARATION CHOICES

This is any specific parameters we have chosen in preparing databases

**ARIBA -> Non coding only**

In [3]:
# CARD

# This code makes the formatted card db ("card_20191023_formatted.fasta") 
# and produces a key for the identifiers in the fasta file "card_20191023_link.csv"
# And finally produces a dictionary which we use downstream "card_20191023_link.csv"

card_formatted_no = 1

card_dict = {}

card_recs = []

with open("card_20191023_link.csv", "w") as f:
    writer = csv.writer(f)
    for key in card_db.keys():
        try:
            # First we extract relavent data
            if card_db[key]['model_type'] == "protein homolog model":
                aro_id = card_db[key]['ARO_id']
                name = card_db[key]["ARO_name"]
                for k in card_db[key]['model_sequences']['sequence'].values():
                    card_prot = k['protein_sequence']['sequence']
                    card_dna = k['dna_sequence']['sequence']
                # This way we automatically keep the second sequence
                # Then we rename things so that each program formatts them properly.
                new_id = "cardnewid_{0}" .format(card_formatted_no)
                card_formatted_no += 1
                card_seq = Seq(card_dna)
                card_rec = SeqRecord(card_seq, id=new_id, description="")
                card_recs.append(card_rec)
                # Now we want to create some stuff for a linking database to compare the two
                orig_id = name + "_ARO"+ aro_id
                card_dict[new_id] = {"orig_id":orig_id, 
                                    "dna_seq":card_dna,
                                    "prot_seq":card_prot}
                writer.writerow([orig_id, new_id, str(card_seq)])
        except:
            # This line removes the 1 entry which just has a description of the database
            pass

SeqIO.write(card_recs, "card_20191023_formatted.fasta", "fasta")

2617

In [4]:
# Resfinder

# Likewise for the resfinder database
# The formatted fasta "resfinder_20191001_formatted.fasta"
# link for ids "resfinder_20191001_link.csv"
# Dictionary for downstream analysis resfinder_db

# Firstly, you concatenate and read in the database
subprocess.check_call("cat resfinder_20191001/*.fsa > resfinder.fasta", shell=True)
resfinder_initdb = SeqIO.parse("resfinder_20191001/resfinder.fasta", "fasta")

# Then you assign a new unique name to each element of the database
# Note some elements are removed (see above) to ensure each DNA sequence only has one name
# The way things are linked are put into a resfinder_20191001_link.csv file
# The final database is then written into a resfinder_20191001_formatted.fasta
resfinder_db = {}
identified_seqs = []
newid = 0
for k in resfinder_initdb:
    if str(k.seq) not in identified_seqs:
        newid += 1
        identified_seqs.append(str(k.seq))
        resfinder_db["resfindernewid_{0}" .format(newid)] = k
out_recs = []
with open("resfinder_20191001_link.csv", "w") as f:
    writer = csv.writer(f, delimiter = ",")
    for k in resfinder_db:
        writer.writerow([k, resfinder_db[k].id, str(resfinder_db[k].seq)])
        k_id = k
        k_desc = ""
        k_seq = resfinder_db[k].seq
        k_rec = SeqRecord(k_seq, id=k_id, description=k_desc)
        out_recs.append(k_rec)

SeqIO.write(out_recs, "resfinder_20191001_formatted.fasta", "fasta")

3079

## <span style="color: blue;">Next comes how to format the databases </span>

This code basically goes through the exact commands we use to format the databases for each program. Note for each one we using singularity images which contain each program. Your actual locations of some script (e.g. the SRST2 specific ones)  may vary

As an additional side point this database then needs to be formatted to meet ABRicate's structure
The script below does this

~~~
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

base_db = SeqIO.parse("<db_file>","fasta")
abricate_recs = []

for k in base_db:
    k_seq = k.seq
    k_id = "resfinderformatted~~~{0}~~~00{1}".format(k.id, "{:04d}" .format(int(k.id.split("_")[-1])))
    k_rec = SeqRecord(k_seq, id=k_id, description="")
    abricate_recs.append(k_rec)
SeqIO.write(abricate_recs, "sequences", "fasta")
~~~


##### Each of the following illustrates the steps required to prep the database

**preparing the ABRicate_db**

~~~
python abricate_format.py 
mkdir abricate_db
mv sequences abricate_db/
cd abricate_db
makeblastdb -in sequences -title <db_title> -dbtype nucl -hash_index
cd ..
~~~
**preparing the ARIBA db**
~~~
ariba prepareref --all_coding no -f <db_file> ariba_db
~~~

**Preparing the KmerResistance db**
** Note for KmerResistance, there is an additional file**
### add to this text ### should primarily been in the supplements

The exception to this was that KmerResistance requires a “bacteria.fsa” FASTA file of all complete genomes in NCBI’s RefSeq database to filter low-coverage matches. As of writing, the version used by KmerResistance’s authors was not publicly available, however we attempted mitigate this using two alterations; replacing the file using a FASTA containing all complete genomes as identified by Centrifuge-download[14], and applying an average depth of coverage cut-off of >5x to KmerResistance results. For ABRicate (which uses BLASTn to search assemblies), assemblies were produced using SPAdes[8] run with default parameters. 

~~~
mkdir kmerres_db
mv bacteria.fsa kmerres_db
cp <db_file> kmerres_db/
cd kmerres_db
kma index -i bacteria.fsa -o bacteria -Sparse ATG
mv <db_file> kmerres_fasta.fa
kma_index -i kmerres_fasta.fa -o kmerres_fasta
cd ..
~~~

**Preparing the SRST2 db**
~~~
mkdir srst2_db
cd srst2_db/
mv <db_file> rawseqs.fasta
cd-hit-est -i rawseqs.fasta -o rawseqs_cdhit90 -d 0 > rawseqs_cdhit90.stdout
python /srst2/database_clustering/cdhit_to_csv.py --cluster_file rawseqs_cdhit90.clstr --infasta rawseqs.fasta --outfile rawseqs_clustered.csv
python /srst2/database_clustering/csv_to_gene_db.py -t rawseqs_clustered.csv -o seqs_clustered.fasta -f rawseqs.fasta -c 4
cd ..
~~~

** A similar method is used for preparing the card and legacy resfinder databases **





## <span style="color: blue;">Comparing the databases</span>

Next step we want to compare the two databases
The resfinder database is now encapsulated in the resfinder_db dictionary
The card database is now encapsuled in card_dict

**Now to compare these two we are going to do the following.**
1. translate resfinder_db sequences
2. compare the protein sequences from the two databases. [note this requires some wiggle room re first and last amino-acids, see above]
3. give each unique protein sequence a unique id.
4. Identify whether DNA variants are the same or different

**Note all elements of each database are disctinct (we have removed duplicates as part of formatting)**


In [9]:

# First we start with a function to compare two proteins with "wiggle room"
def compare_2_prots(s1, s2):
    # Now as there is a bit of a wiggle around what sequence is at 
    s1_poss = [s1,s1[1:],s1[:-1],s1[1:-1]]
    s2_poss = [s2,s2[1:],s2[:-1],s2[1:-1]]
    linked = False
    for k in s1_poss:
        if k in s2_poss:
            linked = True
    return linked

# Next lets get all of the proteins from both the CARD and Resfinder databases
resfinder_prots = {}
for k in resfinder_db:
    resfinder_prots[k] = str(resfinder_db[k].seq.translate(11))

card_prots = {}
for k in card_dict:
    card_prots[k] = card_dict[k]['prot_seq']


# Now we combine these into a list of individual proteins
prot_list = []
for k in resfinder_prots:
    seen_before  = False
    for j in prot_list:
        if compare_2_prots(resfinder_prots[k], j) == True:
            seen_before = True
    if seen_before == False:
        prot_list.append(resfinder_prots[k])
    
for k in card_prots:
    seen_before = False
    for j in prot_list:
        if compare_2_prots(card_prots[k], j) == True:
            seen_before = True
    if seen_before == False:
        prot_list.append(card_prots[k])

#Turning these into a dictionary
prots_full = {}
prot_no = 1
for k in prot_list:
    prot_id = "uprotein_{0}".format(prot_no)
    prots_full[prot_id] = k
    prot_no += 1

# Finally we want to make an output which combines all of these.
# We will produce two however, 
# 1 which only contains the simple names linking (and official names) (CSV) and 1 which contains all of the details about the sequence (JSON)

final_dict = {}
with open("db_comparison.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["prot_id", 'card_fids','card_oids', 'resfinder_fids', 'resfinder_oids'])
    for k in prots_full:
        final_dict[k] ={"card":{}, "resfinder":{}, "protein":prots_full[k]}
        k_card_ids = []
        k_resfinder_ids = []
        for j in card_prots:
            if compare_2_prots(prots_full[k], card_prots[j]) == True:
                k_card_ids.append(j)
        k_card_origid = [card_dict[j]['orig_id'] for j in k_card_ids]
        for j in k_card_ids:
            final_dict[k]['card'][j] = card_dict[j]
        for j in resfinder_prots:
            if compare_2_prots(prots_full[k], resfinder_prots[j]) == True:
                k_resfinder_ids.append(j)
        for j in k_resfinder_ids:
            final_dict[k]['resfinder'][j] = {resfinder_db[j].id:str(resfinder_db[j].seq)}
        k_resfinder_origid = [resfinder_db[j].id for j in k_resfinder_ids]
        csv_line = [k, ":".join(k_card_ids),":".join(k_card_origid), 
                   ":".join(k_resfinder_ids), ":".join(k_resfinder_origid)]
        writer.writerow(csv_line)


with open("db_comparison_full.json", "w") as f:
    x = json.dumps(final_dict)
    json.dump(x, f) 


/Users/timdavies/anaconda/lib/python3.6/site-packages/Bio/Seq.py:2715: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


## <span style="color: blue;">Comparing Resfinder and CARD output</span>

Note I will only investigate differences between the two that I encounter in my data., 
so the next step will be to compare the output between CARD and Resfinder.

This bit needs to be updated, given the figure I intend to produce? compare all full databases.


In [52]:
#############################################################
#               LOADING THE DATA                            #
#############################################################

# Note during the covid outbreak I have created a temporary file structure to enable analysis of files
result_dict = {}


linking_csv = pd.read_csv("db_comparison.csv", index_col=0).fillna("")
card_linkdict = {}
for k in linking_csv.index:
    for j in linking_csv.loc[k].card_fids.split(":"):
        card_linkdict[j] = k
resfinder_linkdict = {}
for k in linking_csv.index:
    for j in linking_csv.loc[k].resfinder_fids.split(":"):
        resfinder_linkdict[j]=k

### DELETE THIS SECTION POST pre-print publication

# this code essentially produces a dictionary where keys are guuids and then within that 
# there is a card and a resfinder reports

card_dir = "../../../result_tarballs/pl_comp_data/db_comparison_result_tarballs/card_results"
card_reports = [os.path.join(root, f) for root, dirs, files in os.walk(card_dir) for f in files]
rf_dir = "../../../result_tarballs/pl_comp_data/db_comparison_result_tarballs/resfinder_results"
rf_reports = [os.path.join(root, f) for root, dirs, files in os.walk(rf_dir) for f in files]
guuids = [k.split("\\")[-1].split("_")[0] for k in rf_reports if "summary" not in k]
result_dict = {k:{"card":"", "resfinder":""} for k in guuids}
for k in result_dict:
    card_fl = [f for f in card_reports if k in f]
    assert len(card_fl) == 1
    card_fl = card_fl[0]
    resfinder_fl = [f for f in rf_reports if k in f]
    assert len(resfinder_fl) == 1 
    resfinder_fl = resfinder_fl[0]
    result_dict[k]["card"] = card_fl
    result_dict[k]["resfinder"] = resfinder_fl
    


### END OF DELETE




In [55]:
class isolate:
    
    def __init__(self, guuid):
        self.guuid = guuid
        self.resfinder_fl = pd.read_csv(result_dict[self.guuid]["resfinder"], "\t")
        self.resfinder_trgs = sorted([resfinder_linkdict[k] for k in list(self.resfinder_fl["GENE"])])
        self.card_fl = pd.read_csv(result_dict[self.guuid]["card"], "\t")
        self.card_trgs = sorted([card_linkdict[k] for k in list(self.card_fl["GENE"])])
        print(self.resfinder_trgs)
        print(self.card_trgs)
        print("")


for g in guuids:
    x = isolate(g)
    


['uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_197', 'uprotein_2426', 'uprotein_2525', 'uprotein_2732', 'uprotein_2761', 'uprotein_2791', 'uprotein_2828', 'uprotein_2949', 'uprotein_508']
['uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_2426', 'uprotein_2730', 'uprotein_2761', 'uprotein_2951', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3111', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3365', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3385', 'uprotein_3386', 'uprotein_3396', 'uprote

['uprotein_2525', 'uprotein_2791']
['uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3580', 'uprotein_3583', 'uprotein_3621', 'uprotein_3622', 'uprotein_3626', 'uprotein_3727', 'uprotein_3728']

['uprotein_2099', 'uprotein_236', 'uprotein_2525', 'uprotein_2749', 'uprotein_2791', 'uprotein_41', 'uprotein_41']
['uprotein_2099


['uprotein_1254', 'uprotein_236', 'uprotein_2525', 'uprotein_2749', 'uprotein_2791', 'uprotein_41', 'uprotein_41']
['uprotein_1254', 'uprotein_237', 'uprotein_237', 'uprotein_2749', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3580', 'uprotein_3583', 'uprotein_3621', 'uprotein_3622', 'uprotein_3626', 'uprotein_3727', '


['uprotein_2525', 'uprotein_2791']
['uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3580', 'uprotein_3580', 'uprotein_3583', 'uprotein_3621', 'uprotein_3622', 'uprotein_3626', 'uprotein_3727', 'uprotein_3728']

['uprotein_115', 'uprotein_1254', 'uprotein_158', 'uprotein_166', 'uprotein_2001', 'uprotein_234', 'uprotein_252

['uprotein_1254', 'uprotein_158', 'uprotein_236', 'uprotein_2525', 'uprotein_2750', 'uprotein_2791', 'uprotein_2815', 'uprotein_2929', 'uprotein_41']
['uprotein_1273', 'uprotein_237', 'uprotein_2749', 'uprotein_2815', 'uprotein_2929', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3427', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3580', 'uprotein_3583', 'uprotein_3621', 

...

['uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_162', 'uprotein_2525', 'uprotein_2566', 'uprotein_2595', 'uprotein_2732', 'uprotein_2828', 'uprotein_2904', 'uprotein_2949', 'uprotein_69', 'uprotein_846']
['uprotein_158', 'uprotein_158', 'uprotein_160', 'uprotein_2566', 'uprotein_2595', 'uprotein_2730', 'uprotein_2934', 'uprotein_2951', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3111', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3255', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3365', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprot


['uprotein_2525', 'uprotein_2791']
['uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3365', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3580', 'uprotein_3583', 'uprotein_3621', 'uprotein_3622', 'uprotein_3626', 'uprotein_3727', 'uprotein_3728']

['uprotein_1164', 'uprotein_1254', 'uprotein_158', 'uprotein_236', 'uprotein_2525', 'uprotein_2

['uprotein_1254', 'uprotein_158', 'uprotein_166', 'uprotein_2525', 'uprotein_2791', 'uprotein_2828', 'uprotein_2910']
['uprotein_1273', 'uprotein_166', 'uprotein_2910', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3111', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3365', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3427', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3583', 'uprotein_3621', 'uprotein_3622', 'uprotein_3626', 'uprotein_3727'


['uprotein_1254', 'uprotein_2525']
['uprotein_1273', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3246', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', 'uprotein_3580', 'uprotein_3583', 'uprotein_3621', 'uprotein_3622', 'uprotein_3626', 'uprotein_3727', 'uprotein_3728']

['uprotein_115', 'uprotein_1254', 'uprotein_158', 'uprotein_2001', 'uprotein_236', 'uprotein_2525', 'uprotein_2732', 'uprotein_27


['uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_2525', 'uprotein_2732', 'uprotein_2815', 'uprotein_2949']
['uprotein_158', 'uprotein_158', 'uprotein_158', 'uprotein_2730', 'uprotein_2815', 'uprotein_2951', 'uprotein_3014', 'uprotein_3020', 'uprotein_3028', 'uprotein_3037', 'uprotein_3089', 'uprotein_3091', 'uprotein_3121', 'uprotein_3123', 'uprotein_3133', 'uprotein_3134', 'uprotein_3148', 'uprotein_3175', 'uprotein_3178', 'uprotein_3186', 'uprotein_3200', 'uprotein_3204', 'uprotein_3209', 'uprotein_3219', 'uprotein_3235', 'uprotein_3246', 'uprotein_3255', 'uprotein_3260', 'uprotein_3261', 'uprotein_3265', 'uprotein_3269', 'uprotein_3286', 'uprotein_3289', 'uprotein_3291', 'uprotein_3292', 'uprotein_3369', 'uprotein_3371', 'uprotein_3382', 'uprotein_3386', 'uprotein_3396', 'uprotein_3403', 'uprotein_3420', 'uprotein_3427', 'uprotein_3432', 'uprotein_3436', 'uprotein_3458', 'uprotein_3485', 'uprotein_3493', 'uprotein_3493', 'uprotein_3527', 'uprotein_3543', '